In [1]:
import torch, os
import pandas as pd
import torch.nn as nn
import numpy as np
from torch import optim, as_tensor
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.autograd import Variable

In [2]:
os.chdir('data/MNIST')

In [3]:
trainData = pd.read_csv("train.csv")

In [4]:
trainLabels = trainData['label']
trainData = trainData.drop(columns = "label")
trainData /= 255

In [5]:
train_tensor = torch.tensor(trainData.values, dtype = torch.double)
label_tensor = torch.tensor(trainLabels.values, dtype = torch.double)
train_data = [(train_tensor[i], label_tensor[i]) for i in range(train_tensor.size()[0])]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], dtype=torch.float64)

In [6]:
class DigNet(nn.Module):
    def __init__(self, input_size, hidden1_size, hidden2_size, num_classes):
        super(DigNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1_size)
        self.sigmoid = nn.LogSigmoid()
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.relu = nn.LogSigmoid()
        self.out = nn.Linear(hidden2_size, num_classes)
        
    def forward(self, x):
#         print(x)
        x = self.fc1(x)
        x = self.sigmoid(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.out(x)
        return x

In [7]:
model = DigNet(784, 128, 64, 10)
model.double()
optimizer = optim.SGD(model.parameters(), lr=0.01)
num_epochs = 20

In [8]:
batch_size = 60

In [15]:
# train_dataset = torch.utils.data.Dataset(train_tensor, label_tensor)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size = batch_size, shuffle=True)
criterion = nn.MultiMarginLoss()

In [16]:
for i in range(num_epochs):
    cum_loss = 0
    for inputs, labels in train_loader:
#         print(inputs)
        optimizer.zero_grad()
#         print(inputs.size())
        output = model(inputs)
#         print(output)
#         output = output.long()
#         print(output)
            
        loss = criterion(output, labels.long())
        loss.backward()
        optimizer.step()
        
        cum_loss += loss.item()
        
    print("Training loss: ", cum_loss / len(train_loader))

Training loss:  0.042853108195381566
Training loss:  0.04133493235258021
Training loss:  0.04010300387760289
Training loss:  0.03925055116147189
Training loss:  0.03839277754496969
Training loss:  0.03774919940877181
Training loss:  0.03703431773681874
Training loss:  0.03654452258394984
Training loss:  0.03598596240489548
Training loss:  0.03543897179744871
Training loss:  0.03501453179254215
Training loss:  0.034586211437259776
Training loss:  0.03421094804778814
Training loss:  0.03384455658176753
Training loss:  0.0335574092631456
Training loss:  0.03317058751516652
Training loss:  0.03285326086903233
Training loss:  0.03259500180563407
Training loss:  0.03234241298730763
Training loss:  0.032068022015047916


In [23]:
with torch.no_grad():
    
        model(train_data[0][0]).max(0)[1].item()

1